In [4]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV 
from sklearn.grid_search import RandomizedSearchCV 

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import pymorphy2
import re

import sys

reload(sys)
sys.setdefaultencoding('utf8')

In [5]:
data = pd.read_csv('./data/data.csv')
del data['Unnamed: 0']

In [6]:
data.head()

,grade,link,text,content
0,5,https://technopoint.ru/product/42db0e393aab333...,"большой экран, мощный аккумулятор, режим для ч...",https://technopoint.ru/product/42db0e393aab333...
1,5,https://technopoint.ru/product/da39313444f8333...,Телефоном пользуюсь немного и это скорее перв...,https://technopoint.ru/product/da39313444f8333...
2,4,https://technopoint.ru/product/44b2f37e4035333...,"Цена, брал за 7900 руб. Яркий экран, хорошо ...",https://technopoint.ru/product/44b2f37e4035333...
3,5,https://technopoint.ru/product/305c1b434efe333...,Цельный качественный корпус с большой батарее...,https://technopoint.ru/product/305c1b434efe333...
4,5,https://technopoint.ru/product/0ecb5f413466333...,"камера, металл корпус , андроид отличный бюдже...",https://technopoint.ru/product/0ecb5f413466333...


In [7]:
data.grade.value_counts()

5    8766
4    2408
3    1060
1     652
2     640
Name: grade, dtype: int64

In [8]:
data['label'] = data.grade.apply(lambda x: 0 if x < 4 else 1)

In [9]:
data.label.value_counts()

1    11174
0     2352
Name: label, dtype: int64

In [20]:
corpus = data[['text', 'label']]

In [21]:
drop_indices = np.random.choice(corpus[corpus.label == 1].index, 8750, replace=False)
corpus = corpus.drop(drop_indices)
corpus.label.value_counts()

1    2424
0    2352
Name: label, dtype: int64

In [22]:
def clean_str(string):
    morph = pymorphy2.MorphAnalyzer()
    
    symbols = [
        ',', '.', '-', '*', '#', ')', '(', '/', '<', '>', ':', '+', '?', '!', '"', '"', '%', '=', '\\', '}'
    ]
    
    for symbol in symbols:
        string = str(string).replace(symbol, ' ')
        
    words = string.split()
    normalized_words = []
    
    for word in words:
        normalized_words.append(morph.parse(unicode(word.strip()))[0].normal_form)
        
    string = unicode(' '.join(normalized_words))
    
    return string

corpus['text'] = corpus.text.apply(clean_str)

In [25]:
corpus.head()

,text,label
0,большой экран мощный аккумулятор режим для чте...,1
1,телефон пользоваться немного и это скорее перв...,1
2,цена брать за 7900 рубль яркий экран хорошо ви...,1
6,пользоваться телефон 4 месяц не однократно лов...,0
7,звук комплект камера материал дизайн характери...,1


In [29]:
cv = StratifiedKFold(corpus.label, n_folds=10, shuffle=True, random_state=1)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='word')),
    ('classifier', LogisticRegression())
])

pipeline_params = {
    'classifier__penalty': ['l1', 'l2'],
    'vectorizer__stop_words': [stopwords.words('russian'), None],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'vectorizer__norm': ['l1', 'l2']
}

grid = GridSearchCV(pipeline, pipeline_params, cv=cv, refit=True, verbose=1, n_jobs=4)

grid.fit(corpus.text.values.astype('U'), corpus.label)
best = grid.best_estimator_
print(
    "Accuracy (TfidfVectorizer + LogisticRegression): {}, params {}" . format(grid.best_score_, grid.best_params_)
)
grid.best_score_

0.9876465661641541

In [57]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 3), analyzer='word' #, stop_words=stopwords.words('russian'),norm='l2'
)
X_vect = vectorizer.fit_transform(corpus.text.values.astype('U'))

classifier = LogisticRegression(penalty='l2')
fitted = classifier.fit(X_vect, corpus.label)

In [60]:
X_test = vectorizer.transform([clean_str('я любительница громкой музыки. Тише телефона у меня еще не было!! Приложений мало, память маленькая. Толком ни чего скачать не можешь. Вечно приходится что то удалять. Бывало такое что сенсор заедал без особых на то причин. Телефону 1,5 месяца. Есть телефоны намного лучше. НЕ БЕРИТЕ !!! Я пожалела...')])
fitted.predict_proba(X_test)

array([[ 0.70440239,  0.29559761]])

In [56]:
fitted.predict(X_test)

array([0])

In [66]:
data_test = pd.read_csv('./data/test_data_to_predict.csv')
data_test['text'] = data_test.text.apply(clean_str)

In [67]:
texts_to_predict = data_test.text.values.astype('U')

In [72]:
X_test = vectorizer.transform(texts_to_predict)
predicts = fitted.predict(X_test)
predicts

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1])